# Good Stock Recommendation by using Machine Learning


## Objective:

Long term investment is one of the major investment strategies. However, calculating intrinsic value of some company and evaluating shares for long term investment is not easy, since analyst have to care about a large number of financial indicators and evaluate them in a right manner.
    
Our goal here is to identify good stocks that are growing healthily at a sustainable pace by using Machine Learning techniques by analyzing their key financial statistic ratio. With this, it is hopefully to reduce investor's effort and time to make tremendous study on annaul and/or quaterly report of each stock.


### 1. Get List of S&P (Standard & Poor's) 500 Companies as of date

In [19]:
#1.1. read the list of S&P 500 companies list from URL https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
import pandas as pd

df_sp500_comp = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', header=0, parse_dates=False)
df_sp500_comp = df_sp500_comp[0]
df_sp500_comp.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


### 2. Download S&P 500 Stock's Latest Key Financial Information from Yahoo Finance

In [4]:
import urllib.request
import os
import time

url_yahoo_fin = "https://finance.yahoo.com/quote/{0}/key-statistics?p={1}"

#create directory to store stock's financial information 
directory_fwd_fin_data = os.path.join(os.getcwd(), 'Forward_Finance_Data_Web_Files') 
if not os.path.exists(directory_fwd_fin_data):
    os.makedirs(directory_fwd_fin_data)

#Run through each S&P 500 companies to download the file
counter = 0
for index, row in df_sp500_comp.iterrows():
    try:
        ticker_symbol = row['Ticker symbol'].replace('.','-')
        url_ticker = url_yahoo_fin.format(ticker_symbol,ticker_symbol)         
        urllib.request.urlretrieve(url_ticker, os.path.join(directory_fwd_fin_data, ticker_symbol+".html"))
        counter+=1
        print(str(counter), ticker_symbol +".html")       
       
    except Exception as e:
        print(str(e))
        time.sleep(100)

1 MMM.html
2 ABT.html
3 ABBV.html
4 ACN.html
5 ATVI.html
6 AYI.html
7 ADBE.html
8 AMD.html
9 AAP.html
10 AES.html
11 AET.html
12 AMG.html
13 AFL.html
14 A.html
15 APD.html
16 AKAM.html
17 ALK.html
18 ALB.html
19 ARE.html
20 ALXN.html
21 ALGN.html
22 ALLE.html
23 AGN.html
24 ADS.html
25 LNT.html
26 ALL.html
27 GOOGL.html
28 GOOG.html
29 MO.html
30 AMZN.html
31 AEE.html
32 AAL.html
33 AEP.html
34 AXP.html
35 AIG.html
36 AMT.html
37 AWK.html
38 AMP.html
39 ABC.html
40 AME.html
41 AMGN.html
42 APH.html
43 APC.html
44 ADI.html
45 ANDV.html
46 ANSS.html
47 ANTM.html
48 AON.html
49 AOS.html
50 APA.html
51 AIV.html
52 AAPL.html
53 AMAT.html
54 APTV.html
55 ADM.html
56 ARNC.html
57 AJG.html
58 AIZ.html
59 T.html
60 ADSK.html
61 ADP.html
62 AZO.html
63 AVB.html
64 AVY.html
65 BHGE.html
66 BLL.html
67 BAC.html
68 BK.html
69 BAX.html
70 BBT.html
71 BDX.html
72 BRK-B.html
73 BBY.html
74 BIIB.html
75 BLK.html
76 HRB.html
77 BA.html
78 BKNG.html
79 BWA.html
80 BXP.html
81 BSX.html
82 BHF.html
83 BMY.

###  3. Function to query Yahoo Finance stock price

In [12]:
#using 3rd party code from https://github.com/c0redumb/yahoo_quote_download to download stock price from Yahoo Finance
import yahoo_quote_download as yqd
from datetime import datetime, timedelta
import pandas as pd

def load_quote(ticker, startDate, endDate, count=0):
    count+=1
#     if startDate.weekday() == 5:
#         startDate = startDate + timedelta(days=2)
#     elif startDate.weekday() == 6:
#         startDate = startDate + timedelta(days=1)
        
#     if endDate.weekday() == 5:
#         endDate = endDate + timedelta(days=2)
#     elif endDate.weekday() == 6:
#         endDate = endDate + timedelta(days=1)
        
    start = startDate.strftime('%Y%m%d')
    end = endDate.strftime('%Y%m%d')
#     print('===', ticker, start, end,'===')
    data = yqd.load_yahoo_quote(ticker, start, end)    
    df = pd.DataFrame([sub.split(",") for sub in data if len(sub.split(",")[0])>0 and sub.split(",")[0] != 'Date'], columns=['Date','Open','High','Low','Close','Adj Close','Volume'])    
      
    if len(df) == 0 and count < 10:
        value = None
        return load_quote(ticker, startDate + timedelta(days=1), endDate + timedelta(days=1), count)
    else:        
        value = df.iloc[0]['Adj Close'] if len(df) > 0 else None
    return value#df

### 3.1 Function to query quandl stock price from WIKI Prices dataset

### Installation

To install Quandl's python package, use the package manager pip inside your terminal. For detail information, refer to https://www.quandl.com/ 

```(python)
$ pip install quandl 
```

In [13]:
import quandl as ql

def load_quote_ql(ticker, startDate, endDate, count=0):
    count+=1
    ql.ApiConfig.api_key = "w3q8uMsYxQgDtpTBfoWh"
    start = startDate.strftime('%Y-%m-%d')
    end = endDate.strftime('%Y-%m-%d')
    #print('===', ticker, start, end,'===')
    df = ql.get("WIKI/" + ticker.upper(), start_date=start, end_date=end)
    #df
    if len(df) == 0 and count < 10:
        value = None
        return load_quote_ql(ticker, startDate + timedelta(days=1), endDate + timedelta(days=1), count)
    else:        
        value = df.iloc[0]['Adj. Close'] if len(df) > 0 else None
        
    return value

### 4. Read historical Key Financial Statistic data from Yahoo Finance web files

In [23]:
#3.1. Read all historical Key Financial Statistic data from Yahoo Finance web files 
# (historical Key Statistic couldn't be retrieve from Yahoo Finance online)
import os
import re
from datetime import datetime
from time import mktime
#import datetime
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

directory_his_fin_data = os.path.join(os.getcwd(), 'Historical_Finance_Data')
directory_pro_his_fin_data = os.path.join(os.getcwd(), 'Processed_Historical_Finance_Data')

key_features_lst = []

key_features = [
        'Market Cap',
        'Enterprise Value ',
        'Trailing P/E',
        'Forward P/E',
        'PEG Ratio',
        'Price/Sales',
        'Price/Book',
        'Enterprise Value/Revenue',
        'Enterprise Value/EBITDA',
        'Profit Margin',
        'Operating Margin',
        'Return on Assets',
        'Return on Equity',
        'Revenue',
        'Revenue Per Share',
        'Gross Profit',
        'EBITDA',
        'Net Income Avl to Common',
        'Diluted EPS',
        'Total Cash',
        'Total Cash Per Share',
        'Total Debt',
        'Total Debt/Equity',
        'Current Ratio',
        'Book Value Per Share',
        'Operating Cash Flow|From Operations',
        'Levered Free Cash Flow|Free Cashflow',
        'Beta',
        '% Held by Insiders',
        '% Held by Institutions',
        'Shares Short',
        'Short Ratio',
        'Short % of Float',
        'Shares Short',
        'Trailing Annual Dividend Rate|Annual Dividend',
        'Trailing Annual Dividend Yield|Dividend Yield'
    ]

#Run through each S&P 500 companies to read the Yahoo Finance offline file
#  **As most of the offline file collected was up to year 2013, and, by using current S&P 500 companies list,
#  **some companies which are newly added into S&P 500 will not get their historical Key Financial Statistic.
#  **In this case, we will ignore analysis of new S&P 500 companies.
counter = 0
for index, row in df_sp500_comp.iterrows():
    try: 
        ticker_symbol = row['Ticker symbol'].replace('.','-')
        ticker_folder_processed_data = os.path.join(directory_pro_his_fin_data, ticker_symbol+'.csv')
        if os.path.exists(ticker_folder_processed_data):
            print("Historical Finance Data already processed for Ticker [{0}]. Skip for next Ticker".format(ticker_symbol))
            continue        
        ticker_folder = os.path.join(directory_his_fin_data, ticker_symbol.lower())
        security = row['Security']
        sector = row['GICS Sector']
        industry = row['GICS Sub Industry']
        
        
        counter+=1
        print(counter)
        print("Reading data for Ticker [{0}]".format(ticker_symbol))
        
        base_stock_price = None
        base_sp500_index = None
        
        if os.path.exists(ticker_folder):
            key_features_lst = []
            for filename in os.listdir(ticker_folder):
                full_file_path = os.path.join(ticker_folder, filename)
              
                null_value_counter = 0
                key_features_value = {}
                
                #get the datetime & unix time of the file which represent the Key Statistic of the date
                date_stamp = datetime.strptime(filename, '%Y%m%d%H%M%S.html')
                unix_time = time.mktime(date_stamp.timetuple())
                
                print("...processing file for date [{0}]...".format(date_stamp))
                
                key_features_value['Date'] = date_stamp
                key_features_value['Unix_Time'] = unix_time
                key_features_value['Ticker'] = ticker_symbol
                key_features_value['Security'] = security
                key_features_value['GICS Sector'] = sector
                key_features_value['GICS Sub Industry'] = industry
                
                html_doc = open(full_file_path, "r").read()
                soup = BeautifulSoup(html_doc, 'lxml')
                
                stock_price = None
                stock_price_change = None
                sp500_index = None
                sp500_index_change = None
                stock_sp500_diff = None
                ranking = None
                
                stock_price = load_quote_ql(ticker_symbol, date_stamp, date_stamp) 
                sp500_index = load_quote("^GSPC", date_stamp, date_stamp)
                
                if base_stock_price == None:
                    #get previous year stock price
                    base_stock_price = load_quote_ql(ticker_symbol, date_stamp-timedelta(days=90), date_stamp-timedelta(days=90)) 
                
                if base_sp500_index == None:
                    #get previous year index price
                    base_sp500_index = load_quote("^GSPC", date_stamp-timedelta(days=90), date_stamp-timedelta(days=90)) 
                    
                if base_stock_price !=  None and stock_price != None:                    
                    stock_price_change = (float(stock_price) - float(base_stock_price))/float(base_stock_price)*100
                else:
                    stock_price_change = 0
                
                if base_sp500_index !=  None and sp500_index != None:                    
                    sp500_index_change = (float(sp500_index) - float(base_sp500_index))/float(base_sp500_index)*100
                else:
                    sp500_index_change = 0
                
                if stock_price_change != None and sp500_index_change != None:
                    stock_sp500_diff = float(stock_price_change) - float(sp500_index_change)
                else:
                    stock_sp500_diff = 0
                
                if stock_sp500_diff != None:
                    if stock_sp500_diff >= 20:
                        ranking = 5
                    elif stock_sp500_diff >= 15 and stock_sp500_diff < 20:
                        ranking = 4
                    elif stock_sp500_diff >= 10 and stock_sp500_diff < 15:
                        ranking = 3
                    elif stock_sp500_diff >= 5 and stock_sp500_diff < 10:
                        ranking = 2
                    elif stock_sp500_diff >= 0 and stock_sp500_diff < 5:
                        ranking = 1
                    elif stock_sp500_diff > -5 and stock_sp500_diff < 0:
                        ranking = -1
                    elif stock_sp500_diff > -10 and stock_sp500_diff <= -5:
                        ranking = -2
                    elif stock_sp500_diff > -15 and stock_sp500_diff <= -10:
                        ranking = -3
                    elif stock_sp500_diff > -20 and stock_sp500_diff <= -15:
                        ranking = -4
                    elif stock_sp500_diff <= -20:
                        ranking = -5     
                        
                base_stock_price = stock_price
                base_sp500_index = sp500_index
                
                key_features_value['Stock Price'] = stock_price
                key_features_value['Stock Price Change'] = stock_price_change
                key_features_value['S&P 500 Index'] = sp500_index
                key_features_value['S&P 500 Index Change'] = sp500_index_change
                key_features_value['Stock Price vs S&P 500 Index'] = stock_sp500_diff
                key_features_value['Ranking'] = ranking
                    
          
                #retrieve value of Key Statistic  
                for key in key_features:
                    value = None
                    tag = soup.find("td", string=re.compile("^" + key + "+"))                   
                    if tag != None:      
                        value = tag.next_sibling.string.strip().replace(",","").replace(" ", "").replace("N/A","").upper() 
                    else:
                        for tag in soup.find_all("td", class_="yfnc_tablehead1"):                            
                            this_tag = tag.find(string=re.compile("^" + key + "+"))
                            if this_tag != None:
                                value = tag.find_next_sibling("td").string.strip().replace(",","").replace(" ", "").replace("N/A","").upper()
                                break
                     
                    if value != None: 
                        if "B" in value:
                            value = float(value.replace("B",""))*1000000000
                        elif "M" in value:
                            value = float(value.replace("M",""))*1000000
                        elif "K" in value:
                            value = float(value.replace("K",""))*1000
                        elif "%" in value:
                            value = float(value.replace("%",""))/100
                        elif value == "":
                            value = None
                        else:
                            value = float(value)
                    elif value == '':
                        value = None                   
                    
                    if value == None:
                        null_value_counter += 1

                    key_features_value[key] = value
               
                if null_value_counter <= 5:
                    key_features_lst.append(key_features_value.copy()) 
              
            df = pd.DataFrame([{
                    'Ranking':key_features_value.get('Ranking'),
                    ###########
                    'Date':key_features_value.get('Date'),
                    'Unix_Time':key_features_value.get('Unix_Time'),
                    'Ticker':key_features_value.get('Ticker'),
                    'Security':key_features_value.get('Security'),
                    'GICS Sector':key_features_value.get('GICS Sector'),
                    'GICS Sub Industry':key_features_value.get('GICS Sub Industry'),
                    'Stock Price':key_features_value.get('Stock Price'),
                    'Stock Price Change':key_features_value.get('Stock Price Change'),
                    'S&P 500 Index':key_features_value.get('S&P 500 Index'),
                    'S&P 500 Index Change':key_features_value.get('S&P 500 Index Change'),
                    'Stock Price vs S&P 500 Index':key_features_value.get('Stock Price vs S&P 500 Index'),
                    #############
                    'Market Cap':key_features_value.get('Market Cap'),
                    'Enterprise Value':key_features_value.get('Enterprise Value '),
                    'Trailing P/E':key_features_value.get('Trailing P/E'),
                    'Forward P/E':key_features_value.get('Forward P/E'),
                    'PEG Ratio':key_features_value.get('PEG Ratio'),
                    'Price/Sales':key_features_value.get('Price/Sales'),
                    'Price/Book':key_features_value.get('Price/Book'),
                    'Enterprise Value/Revenue':key_features_value.get('Enterprise Value/Revenue'),
                    'Enterprise Value/EBITDA':key_features_value.get('Enterprise Value/EBITDA'),
                    'Profit Margin':key_features_value.get('Profit Margin'),
                    'Operating Margin':key_features_value.get('Operating Margin'),
                    'Return on Assets':key_features_value.get('Return on Assets'),
                    'Return on Equity':key_features_value.get('Return on Equity'),
                    'Revenue':key_features_value.get('Revenue'),
                    'Revenue Per Share':key_features_value.get('Revenue Per Share'),
                    'Gross Profit':key_features_value.get('Gross Profit'),
                    'EBITDA':key_features_value.get('EBITDA'),
                    'Net Income Avl to Common':key_features_value.get('Net Income Avl to Common'),
                    'Diluted EPS':key_features_value.get('Diluted EPS'),
                    'Total Cash':key_features_value.get('Total Cash'),
                    'Total Cash Per Share':key_features_value.get('Total Cash Per Share'),
                    'Total Debt':key_features_value.get('Total Debt'),
                    'Total Debt/Equity':key_features_value.get('Total Debt/Equity'),
                    'Current Ratio':key_features_value.get('Current Ratio'),
                    'Book Value Per Share':key_features_value.get('Book Value Per Share'),
                    'Operating Cash Flow':key_features_value.get('Operating Cash Flow|From Operations'),
                    'Levered Free Cash Flow':key_features_value.get('Levered Free Cash Flow|Free Cashflow'),
                    'Beta':key_features_value.get('Beta'),
                    '% Held by Insiders':key_features_value.get('% Held by Insiders'),
                    '% Held by Institutions':key_features_value.get('% Held by Institutions'),
                    'Shares Short':key_features_value.get('Shares Short'),
                    'Short Ratio':key_features_value.get('Short Ratio'),
                    'Short % of Float':key_features_value.get('Short % of Float'),
                    'Shares Short':key_features_value.get('Shares Short'),
                    'Trailing Annual Dividend Rate':key_features_value.get('Trailing Annual Dividend Rate|Annual Dividend'),
                    'Trailing Annual Dividend Yield':key_features_value.get('Trailing Annual Dividend Yield|Dividend Yield')
                } for key_features_value in key_features_lst],
                columns = [
                        'Ranking',
                        ##########
                        'Date',
                        'Unix_Time',
                        'Ticker',
                        'Security',
                        'GICS Sector',
                        'GICS Sub Industry',
                        'Stock Price',
                        'Stock Price Change',
                        'S&P 500 Index',
                        'S&P 500 Index Change',
                        'Stock Price vs S&P 500 Index',
                        ############
                        'Market Cap',
                        'Enterprise Value',
                        'Trailing P/E',
                        'Forward P/E',
                        'PEG Ratio',
                        'Price/Sales',
                        'Price/Book',
                        'Enterprise Value/Revenue',
                        'Enterprise Value/EBITDA',
                        'Profit Margin',
                        'Operating Margin',
                        'Return on Assets',
                        'Return on Equity',
                        'Revenue',
                        'Revenue Per Share',
                        'Gross Profit',
                        'EBITDA',
                        'Net Income Avl to Common',
                        'Diluted EPS',
                        'Total Cash',
                        'Total Cash Per Share',
                        'Total Debt',
                        'Total Debt/Equity',
                        'Current Ratio',
                        'Book Value Per Share',
                        'Operating Cash Flow',
                        'Levered Free Cash Flow',
                        'Beta',
                        '% Held by Insiders',
                        '% Held by Institutions',
                        'Shares Short',
                        'Short Ratio',
                        'Short % of Float',
                        'Shares Short',
                        'Trailing Annual Dividend Rate',
                        'Trailing Annual Dividend Yield'
                    ])
        
            df.to_csv(ticker_folder_processed_data)
            print("{0} saved!".format(ticker_folder_processed_data))
        
        else:
            print("[{0}-{1}] not a S&P 500 company previously!".format(row['Ticker symbol'].replace('.','-'), row['Security']))
  
    except Exception as e:
        print(str(e))   



1
Reading data for Ticker [MMM]
...processing file for date [2003-12-15 23:43:22]...
...processing file for date [2004-06-03 01:36:52]...
...processing file for date [2004-09-19 22:39:39]...
...processing file for date [2004-12-04 14:49:12]...
...processing file for date [2004-12-10 14:23:38]...
...processing file for date [2005-02-07 03:06:21]...
...processing file for date [2005-03-07 02:24:13]...
...processing file for date [2006-06-16 17:32:12]...
...processing file for date [2006-07-18 01:42:46]...
...processing file for date [2006-08-22 00:52:28]...
...processing file for date [2006-08-29 19:24:21]...
...processing file for date [2006-12-06 03:06:51]...
...processing file for date [2006-12-08 16:29:04]...
...processing file for date [2006-12-13 05:29:35]...
...processing file for date [2006-12-18 15:58:47]...
...processing file for date [2006-12-31 20:04:33]...
...processing file for date [2007-01-07 09:29:13]...
...processing file for date [2007-01-11 23:29:49]...
...processing 

...processing file for date [2007-10-18 06:04:41]...
...processing file for date [2008-01-23 23:45:57]...
...processing file for date [2008-12-04 14:30:57]...
...processing file for date [2008-12-05 00:15:55]...
...processing file for date [2008-12-17 11:42:55]...
...processing file for date [2008-12-20 21:49:14]...
...processing file for date [2011-04-24 04:02:44]...
...processing file for date [2011-08-05 04:11:46]...
...processing file for date [2011-09-25 13:42:54]...
...processing file for date [2011-10-18 13:47:26]...
...processing file for date [2012-03-24 12:13:07]...
...processing file for date [2012-07-02 04:31:22]...
C:\Users\clau3\Desktop\EDS4\Project\Processed_Historical_Finance_Data\ADBE.csv saved!
8
Reading data for Ticker [AMD]
...processing file for date [2003-10-02 17:03:24]...
...processing file for date [2003-12-15 23:11:56]...
...processing file for date [2004-07-16 05:32:42]...
...processing file for date [2004-08-10 02:49:35]...
...processing file for date [2004-

...processing file for date [2013-05-15 13:09:58]...
C:\Users\clau3\Desktop\EDS4\Project\Processed_Historical_Finance_Data\APD.csv saved!
16
Reading data for Ticker [AKAM]
...processing file for date [2004-01-15 08:25:52]...
...processing file for date [2004-04-14 22:31:23]...
...processing file for date [2004-06-20 08:20:55]...
...processing file for date [2004-08-22 18:34:11]...
...processing file for date [2004-10-14 00:44:39]...
...processing file for date [2005-04-05 00:26:53]...
...processing file for date [2006-02-18 09:23:31]...
...processing file for date [2006-04-07 10:36:37]...
...processing file for date [2006-04-12 06:24:50]...
...processing file for date [2006-06-29 20:45:30]...
...processing file for date [2006-11-30 09:22:43]...
...processing file for date [2007-01-13 14:46:03]...
...processing file for date [2007-02-11 18:36:34]...
...processing file for date [2007-02-19 17:28:06]...
...processing file for date [2007-03-08 17:09:23]...
...processing file for date [2007

...processing file for date [2012-03-13 09:39:10]...
...processing file for date [2012-03-16 01:19:55]...
...processing file for date [2012-05-18 20:33:27]...
...processing file for date [2012-07-16 18:22:38]...
...processing file for date [2012-10-21 01:32:38]...
...processing file for date [2013-01-16 10:43:29]...
...processing file for date [2013-01-23 12:04:31]...
...processing file for date [2013-05-16 17:00:56]...
...processing file for date [2013-08-18 08:54:27]...
C:\Users\clau3\Desktop\EDS4\Project\Processed_Historical_Finance_Data\GOOG.csv saved!
29
Reading data for Ticker [MO]
...processing file for date [2004-08-16 22:37:20]...
...processing file for date [2004-10-20 14:13:31]...
...processing file for date [2005-01-13 07:12:16]...
...processing file for date [2005-01-19 17:22:22]...
...processing file for date [2005-03-29 20:56:48]...
...processing file for date [2005-04-04 09:46:14]...
...processing file for date [2005-09-01 05:47:49]...
...processing file for date [2006-

...processing file for date [2007-06-08 15:38:27]...
...processing file for date [2007-10-23 22:08:34]...
...processing file for date [2008-01-23 16:12:37]...
...processing file for date [2008-10-28 18:29:50]...
...processing file for date [2008-12-18 21:41:22]...
...processing file for date [2009-02-01 18:00:03]...
...processing file for date [2009-03-28 14:43:57]...
...processing file for date [2011-04-30 18:31:35]...
...processing file for date [2011-09-03 01:04:10]...
...processing file for date [2011-11-02 21:07:51]...
...processing file for date [2011-11-04 15:56:14]...
...processing file for date [2012-01-02 19:28:59]...
...processing file for date [2012-03-04 02:57:58]...
...processing file for date [2012-03-07 01:40:10]...
...processing file for date [2012-03-07 17:08:49]...
...processing file for date [2012-07-30 22:27:11]...
...processing file for date [2012-08-06 08:14:18]...
...processing file for date [2013-02-06 07:01:26]...
...processing file for date [2013-04-25 05:50:

...

...processing file for date [2007-10-25 06:56:29]...
...processing file for date [2008-01-23 15:43:32]...
...processing file for date [2009-02-08 22:13:32]...
...processing file for date [2009-06-19 19:55:20]...
...processing file for date [2011-04-25 19:52:13]...
...processing file for date [2011-07-24 01:58:05]...
...processing file for date [2011-09-26 08:27:42]...
...processing file for date [2011-11-15 08:56:50]...
...processing file for date [2012-01-12 13:31:37]...
...processing file for date [2012-03-21 20:43:10]...
...processing file for date [2012-04-21 05:22:32]...
...processing file for date [2012-06-25 18:44:41]...
...processing file for date [2013-02-06 08:51:52]...
...processing file for date [2013-04-23 09:33:52]...
...processing file for date [2013-05-16 08:18:14]...
C:\Users\clau3\Desktop\EDS4\Project\Processed_Historical_Finance_Data\UTX.csv saved!
466
Reading data for Ticker [UHS]
[UHS-Universal Health Services, Inc.] not a S&P 500 company previously!
467
Reading da

...processing file for date [2004-11-27 22:18:08]...
...processing file for date [2005-01-21 02:11:30]...
...processing file for date [2005-05-01 07:04:47]...
...processing file for date [2013-04-22 06:14:23]...
...processing file for date [2013-05-07 08:34:37]...
C:\Users\clau3\Desktop\EDS4\Project\Processed_Historical_Finance_Data\VIAB.csv saved!
477
Reading data for Ticker [V]
...processing file for date [2004-04-27 16:22:29]...
...processing file for date [2004-06-21 19:16:21]...
...processing file for date [2004-08-03 21:07:29]...
...processing file for date [2005-01-24 04:31:15]...
...processing file for date [2005-05-26 08:47:49]...
...processing file for date [2006-11-15 08:27:43]...
...processing file for date [2006-11-15 21:41:33]...
...processing file for date [2006-12-11 08:18:33]...
...processing file for date [2007-04-11 05:38:54]...
...processing file for date [2007-06-14 12:50:27]...
...processing file for date [2007-08-16 07:50:46]...
...processing file for date [2008-

...processing file for date [2007-05-11 11:31:48]...
...processing file for date [2007-05-12 13:06:53]...
...processing file for date [2007-05-26 19:57:44]...
...processing file for date [2007-06-03 12:29:55]...
...processing file for date [2007-06-12 15:10:39]...
...processing file for date [2007-07-04 03:46:18]...
...processing file for date [2007-07-13 20:30:32]...
...processing file for date [2007-10-21 13:19:43]...
...processing file for date [2008-01-23 16:21:27]...
...processing file for date [2008-05-11 08:19:15]...
...processing file for date [2008-11-01 18:02:21]...
...processing file for date [2008-12-23 21:59:02]...
...processing file for date [2009-02-06 23:50:18]...
...processing file for date [2010-01-05 20:39:30]...
...processing file for date [2011-09-03 07:59:12]...
...processing file for date [2011-10-11 04:36:45]...
...processing file for date [2011-11-02 23:39:05]...
...processing file for date [2012-01-03 13:50:51]...
...processing file for date [2012-03-16 04:36:

...processing file for date [2013-05-07 08:32:23]...
C:\Users\clau3\Desktop\EDS4\Project\Processed_Historical_Finance_Data\WY.csv saved!
492
Reading data for Ticker [WHR]
...processing file for date [2003-12-15 02:10:26]...
...processing file for date [2005-01-31 19:24:36]...
...processing file for date [2005-02-05 05:51:36]...
...processing file for date [2006-04-22 09:15:29]...
...processing file for date [2007-02-21 20:21:46]...
...processing file for date [2008-02-01 02:03:28]...
...processing file for date [2008-05-04 13:45:10]...
...processing file for date [2009-02-12 19:44:47]...
...processing file for date [2011-11-03 06:06:56]...
...processing file for date [2012-03-19 03:13:18]...
...processing file for date [2013-04-25 00:13:04]...
...processing file for date [2013-05-17 17:56:44]...
C:\Users\clau3\Desktop\EDS4\Project\Processed_Historical_Finance_Data\WHR.csv saved!
493
Reading data for Ticker [WMB]
...processing file for date [2003-12-14 15:50:00]...
...processing file fo

...processing file for date [2013-05-11 23:04:07]...
C:\Users\clau3\Desktop\EDS4\Project\Processed_Historical_Finance_Data\YUM.csv saved!
503
Reading data for Ticker [ZBH]
[ZBH-Zimmer Biomet Holdings] not a S&P 500 company previously!
504
Reading data for Ticker [ZION]
...processing file for date [2004-01-20 04:35:05]...
...processing file for date [2004-06-21 23:15:41]...
...processing file for date [2004-08-11 09:53:59]...
...processing file for date [2004-12-09 08:13:59]...
...processing file for date [2005-02-10 03:31:51]...
...processing file for date [2005-04-05 07:30:49]...
...processing file for date [2005-05-28 03:41:53]...
...processing file for date [2007-04-06 05:19:23]...
...processing file for date [2008-06-20 00:12:51]...
...processing file for date [2009-08-19 01:12:46]...
...processing file for date [2009-11-30 00:39:56]...
...processing file for date [2009-12-13 03:22:31]...
...processing file for date [2011-04-30 18:43:35]...
...processing file for date [2011-09-28 0

### 5. Combine all Key Financial Statistic data for each company into one single csv file

In [4]:
#append all historical Key Financial Statistic data from each Excel into single & final Excel file
import os
import pandas as pd
from datetime import datetime

data_path = os.path.join(os.path.join(os.getcwd(), 'Data'), "Historical.csv")
directory_pro_his_fin_data = os.path.join(os.getcwd(), 'Processed_Historical_Finance_Data')

if os.path.exists(data_path):
    os.remove(data_path)

with open(data_path, 'a+') as f:
    if os.path.exists(directory_pro_his_fin_data):   
        counter = 0
        for filename in os.listdir(directory_pro_his_fin_data):
            full_file_path = os.path.join(directory_pro_his_fin_data, filename)
            df = pd.read_csv(full_file_path).iloc[:,1:]
            df.to_csv(f, index=False, header=True if counter == 0 else False)
            counter+=1      
        
        print('Total {0} files combined and save to {1}'.format(counter,data_path))

Total 388 files combined and save to C:\Users\Lau Chong Ee\Desktop\EDS4\Project\Data\Historical.csv


### 6. Patching stock and S&P500 price if needed

In [14]:
import os
import pandas as pd
from datetime import datetime

directory_pro_his_fin_data = os.path.join(os.getcwd(), 'Repair')
for filename in os.listdir(directory_pro_his_fin_data):
    full_file_path = os.path.join(directory_pro_his_fin_data, filename)
    df = pd.read_csv(full_file_path)
            
    base_stock_price = None
    base_sp500_index = None
    #refill missing stock price data if any
    for index, row in df.iterrows():
        stock_price = None
        stock_price_change = None
        sp500_index = None
        sp500_index_change = None
        stock_sp500_diff = None
        ranking = None

        date_stamp = datetime.strptime(row['Date'], '%Y-%m-%d %H:%M:%S')
        ticker_symbol = row['Ticker']
        stock_price = load_quote(ticker_symbol, date_stamp, date_stamp) #load_quote_ql(ticker_symbol, date_stamp, date_stamp) 
        sp500_index = load_quote("^GSPC", date_stamp, date_stamp)

        print('{0} at date {1}-Stock Price:{2}, S&P Price:{3}'.format(ticker_symbol,date_stamp,stock_price,sp500_index))

    #                 if stock_price == None:
    #                     stock_price = load_quote_ql(ticker_symbol, date_stamp, date_stamp)

        if base_stock_price == None:
            #get previous year stock price
            #base_stock_price = load_quote_ql(ticker_symbol, date_stamp-timedelta(days=90), date_stamp-timedelta(days=90)) 
            base_stock_price = load_quote(ticker_symbol, date_stamp-timedelta(days=30), date_stamp-timedelta(days=30)) 

        if base_sp500_index == None:
            #get previous year index price
            #base_sp500_index = load_quote("^GSPC", date_stamp-timedelta(days=90), date_stamp-timedelta(days=90)) 
            base_sp500_index = load_quote("^GSPC", date_stamp-timedelta(days=30), date_stamp-timedelta(days=30))

        if base_stock_price !=  None and stock_price != None:                    
            stock_price_change = (float(stock_price) - float(base_stock_price))/float(base_stock_price)*100
        else:
            stock_price_change = 0

        if base_sp500_index !=  None and sp500_index != None:                    
            sp500_index_change = (float(sp500_index) - float(base_sp500_index))/float(base_sp500_index)*100
        else:
            sp500_index_change = 0

        if stock_price_change != None and sp500_index_change != None:
            stock_sp500_diff = float(stock_price_change) - float(sp500_index_change)
        else:
            stock_sp500_diff = 0

        if stock_sp500_diff != None:
            if stock_sp500_diff >= 20:
                ranking = 5
            elif stock_sp500_diff >= 15 and stock_sp500_diff < 20:
                ranking = 4
            elif stock_sp500_diff >= 10 and stock_sp500_diff < 15:
                ranking = 3
            elif stock_sp500_diff >= 5 and stock_sp500_diff < 10:
                ranking = 2
            elif stock_sp500_diff >= 0 and stock_sp500_diff < 5:
                ranking = 1
            elif stock_sp500_diff > -5 and stock_sp500_diff < 0:
                ranking = -1
            elif stock_sp500_diff > -10 and stock_sp500_diff <= -5:
                ranking = -2
            elif stock_sp500_diff > -15 and stock_sp500_diff <= -10:
                ranking = -3
            elif stock_sp500_diff > -20 and stock_sp500_diff <= -15:
                ranking = -4
            elif stock_sp500_diff <= -20:
                ranking = -5     

        base_stock_price = stock_price
        base_sp500_index = sp500_index

        row['Stock Price'] = stock_price
        row['Stock Price Change'] = stock_price_change
        row['S&P 500 Index'] = sp500_index
        row['S&P 500 Index Change'] = sp500_index_change
        row['Stock Price vs S&P 500 Index'] = stock_sp500_diff
        row['Ranking'] = ranking                    

    df.to_csv(full_file_path, index=False, header=True)
      


=== CF 20040825 20040825 ===
=== CF 20040826 20040826 ===
=== CF 20040827 20040827 ===
=== CF 20040830 20040830 ===
=== CF 20040831 20040831 ===
=== CF 20040901 20040901 ===
=== CF 20040902 20040902 ===
=== CF 20040903 20040903 ===
=== CF 20040906 20040906 ===
=== CF 20040907 20040907 ===
=== ^GSPC 20040825 20040825 ===
CF at date 2004-08-25 21:01:02-Stock Price:None, S&P Price:1096.189941
=== CF 20040726 20040726 ===
=== CF 20040727 20040727 ===
=== CF 20040728 20040728 ===
=== CF 20040729 20040729 ===
=== CF 20040730 20040730 ===
=== CF 20040802 20040802 ===
=== CF 20040803 20040803 ===
=== CF 20040804 20040804 ===
=== CF 20040805 20040805 ===
=== CF 20040806 20040806 ===
=== ^GSPC 20040726 20040726 ===
=== CF 20060626 20060626 ===
=== ^GSPC 20060626 20060626 ===
CF at date 2006-06-24 17:19:06-Stock Price:1.746136, S&P Price:1250.560059
=== CF 20060525 20060525 ===
=== CF 20061023 20061023 ===
=== ^GSPC 20061023 20061023 ===
CF at date 2006-10-23 12:23:40-Stock Price:2.359261, S&P Pr

=== ^GSPC 20040521 20040521 ===
DG at date 2004-05-21 12:48:29-Stock Price:None, S&P Price:1089.189941
=== DG 20040421 20040421 ===
=== DG 20040422 20040422 ===
=== DG 20040423 20040423 ===
=== DG 20040426 20040426 ===
=== DG 20040427 20040427 ===
=== DG 20040428 20040428 ===
=== DG 20040429 20040429 ===
=== DG 20040430 20040430 ===
=== DG 20040503 20040503 ===
=== DG 20040504 20040504 ===
=== ^GSPC 20040421 20040421 ===
=== DG 20040920 20040920 ===
=== DG 20040921 20040921 ===
=== DG 20040922 20040922 ===
=== DG 20040923 20040923 ===
=== DG 20040924 20040924 ===
=== DG 20040927 20040927 ===
=== DG 20040928 20040928 ===
=== DG 20040929 20040929 ===
=== DG 20040930 20040930 ===
=== DG 20041001 20041001 ===
=== ^GSPC 20040920 20040920 ===
DG at date 2004-09-18 12:11:17-Stock Price:None, S&P Price:1122.199951
=== DG 20040819 20040819 ===
=== DG 20040820 20040820 ===
=== DG 20040823 20040823 ===
=== DG 20040824 20040824 ===
=== DG 20040825 20040825 ===
=== DG 20040826 20040826 ===
=== DG 2

=== ^GSPC 20130423 20130423 ===
DG at date 2013-04-23 16:01:35-Stock Price:49.713009, S&P Price:1562.500000
=== DG 20130520 20130520 ===
=== ^GSPC 20130520 20130520 ===
DG at date 2013-05-18 11:06:51-Stock Price:51.840942, S&P Price:1666.290039
=== GM 20040621 20040621 ===
=== GM 20040622 20040622 ===
=== GM 20040623 20040623 ===
=== GM 20040624 20040624 ===
=== GM 20040625 20040625 ===
=== GM 20040628 20040628 ===
=== GM 20040629 20040629 ===
=== GM 20040630 20040630 ===
=== GM 20040701 20040701 ===
=== GM 20040702 20040702 ===
=== ^GSPC 20040621 20040621 ===
GM at date 2004-06-21 00:28:52-Stock Price:None, S&P Price:1130.300049
=== GM 20040524 20040524 ===
=== GM 20040525 20040525 ===
=== GM 20040526 20040526 ===
=== GM 20040527 20040527 ===
=== GM 20040528 20040528 ===
=== GM 20040531 20040531 ===
=== GM 20040601 20040601 ===
=== GM 20040602 20040602 ===
=== GM 20040603 20040603 ===
=== GM 20040604 20040604 ===
=== ^GSPC 20040524 20040524 ===
=== GM 20040706 20040706 ===
=== GM 2004

=== GM 20060718 20060718 ===
=== GM 20060719 20060719 ===
=== GM 20060720 20060720 ===
=== GM 20060721 20060721 ===
=== GM 20060724 20060724 ===
=== GM 20060725 20060725 ===
=== GM 20060726 20060726 ===
=== GM 20060727 20060727 ===
=== GM 20060728 20060728 ===
=== GM 20060822 20060822 ===
=== GM 20060823 20060823 ===
=== GM 20060824 20060824 ===
=== GM 20060825 20060825 ===
=== GM 20060828 20060828 ===
=== GM 20060829 20060829 ===
=== GM 20060830 20060830 ===
=== GM 20060831 20060831 ===
=== GM 20060901 20060901 ===
=== GM 20060904 20060904 ===
=== ^GSPC 20060822 20060822 ===
GM at date 2006-08-22 19:30:20-Stock Price:None, S&P Price:1297.520020
=== GM 20060724 20060724 ===
=== GM 20060725 20060725 ===
=== GM 20060726 20060726 ===
=== GM 20060727 20060727 ===
=== GM 20060728 20060728 ===
=== GM 20060731 20060731 ===
=== GM 20060801 20060801 ===
=== GM 20060802 20060802 ===
=== GM 20060803 20060803 ===
=== GM 20060804 20060804 ===
=== GM 20061128 20061128 ===
=== GM 20061129 20061129 ==

=== ^GSPC 20120305 20120305 ===
GM at date 2012-03-03 20:20:49-Stock Price:21.840685, S&P Price:1364.329956
=== GM 20120306 20120306 ===
=== ^GSPC 20120306 20120306 ===
GM at date 2012-03-06 16:04:31-Stock Price:21.840685, S&P Price:1364.329956
=== GM 20120806 20120806 ===
=== ^GSPC 20120806 20120806 ===
GM at date 2012-08-06 08:47:02-Stock Price:16.674528, S&P Price:1394.229980
=== GM 20121015 20121015 ===
=== ^GSPC 20121015 20121015 ===
GM at date 2012-10-15 14:50:37-Stock Price:20.563847, S&P Price:1440.130005
=== GM 20130312 20130312 ===
=== ^GSPC 20130312 20130312 ===
GM at date 2013-03-12 02:42:28-Stock Price:23.781147, S&P Price:1556.219971
=== GM 20130521 20130521 ===
=== ^GSPC 20130521 20130521 ===
GM at date 2013-05-21 07:46:14-Stock Price:28.308893, S&P Price:1666.290039
=== KMI 20031215 20031215 ===
=== KMI 20031216 20031216 ===
=== KMI 20031217 20031217 ===
=== KMI 20031218 20031218 ===
=== KMI 20031219 20031219 ===
=== KMI 20031222 20031222 ===
=== KMI 20031223 20031223 =

=== NWSA 20110809 20110809 ===
=== NWSA 20110810 20110810 ===
=== NWSA 20110811 20110811 ===
=== ^GSPC 20110729 20110729 ===
NWSA at date 2011-07-29 11:12:26-Stock Price:None, S&P Price:1300.670044
=== NWSA 20110629 20110629 ===
=== NWSA 20110630 20110630 ===
=== NWSA 20110701 20110701 ===
=== NWSA 20110704 20110704 ===
=== NWSA 20110705 20110705 ===
=== NWSA 20110706 20110706 ===
=== NWSA 20110707 20110707 ===
=== NWSA 20110708 20110708 ===
=== NWSA 20110711 20110711 ===
=== NWSA 20110712 20110712 ===
=== NWSA 20111107 20111107 ===
=== NWSA 20111108 20111108 ===
=== NWSA 20111109 20111109 ===
=== NWSA 20111110 20111110 ===
=== NWSA 20111111 20111111 ===
=== NWSA 20111114 20111114 ===
=== NWSA 20111115 20111115 ===
=== NWSA 20111116 20111116 ===
=== NWSA 20111117 20111117 ===
=== NWSA 20111118 20111118 ===
=== ^GSPC 20111107 20111107 ===
NWSA at date 2011-11-07 22:36:54-Stock Price:None, S&P Price:1261.119995
=== NWSA 20111010 20111010 ===
=== NWSA 20111011 20111011 ===
=== NWSA 201110

=== UA 20100211 20100211 ===
=== UA 20100212 20100212 ===
=== UA 20100215 20100215 ===
=== UA 20100309 20100309 ===
=== UA 20100310 20100310 ===
=== UA 20100311 20100311 ===
=== UA 20100312 20100312 ===
=== UA 20100315 20100315 ===
=== UA 20100316 20100316 ===
=== UA 20100317 20100317 ===
=== UA 20100318 20100318 ===
=== UA 20100319 20100319 ===
=== UA 20100322 20100322 ===
=== ^GSPC 20100309 20100309 ===
UA at date 2010-03-09 13:25:32-Stock Price:None, S&P Price:1138.500000
=== UA 20100208 20100208 ===
=== UA 20100209 20100209 ===
=== UA 20100210 20100210 ===
=== UA 20100211 20100211 ===
=== UA 20100212 20100212 ===
=== UA 20100215 20100215 ===
=== UA 20100216 20100216 ===
=== UA 20100217 20100217 ===
=== UA 20100218 20100218 ===
=== UA 20100219 20100219 ===
=== UA 20110926 20110926 ===
=== UA 20110927 20110927 ===
=== UA 20110928 20110928 ===
=== UA 20110929 20110929 ===
=== UA 20110930 20110930 ===
=== UA 20111003 20111003 ===
=== UA 20111004 20111004 ===
=== UA 20111005 20111005 ==

=== VIAB 20040603 20040603 ===
=== VIAB 20041015 20041015 ===
=== VIAB 20041018 20041018 ===
=== VIAB 20041019 20041019 ===
=== VIAB 20041020 20041020 ===
=== VIAB 20041021 20041021 ===
=== VIAB 20041022 20041022 ===
=== VIAB 20041025 20041025 ===
=== VIAB 20041026 20041026 ===
=== VIAB 20041027 20041027 ===
=== VIAB 20041028 20041028 ===
=== ^GSPC 20041015 20041015 ===
VIAB at date 2004-10-15 12:12:25-Stock Price:None, S&P Price:1103.290039
=== VIAB 20040915 20040915 ===
=== VIAB 20040916 20040916 ===
=== VIAB 20040917 20040917 ===
=== VIAB 20040920 20040920 ===
=== VIAB 20040921 20040921 ===
=== VIAB 20040922 20040922 ===
=== VIAB 20040923 20040923 ===
=== VIAB 20040924 20040924 ===
=== VIAB 20040927 20040927 ===
=== VIAB 20040928 20040928 ===
=== VIAB 20041129 20041129 ===
=== VIAB 20041130 20041130 ===
=== VIAB 20041201 20041201 ===
=== VIAB 20041202 20041202 ===
=== VIAB 20041203 20041203 ===
=== VIAB 20041206 20041206 ===
=== VIAB 20041207 20041207 ===
=== VIAB 20041208 20041208 

KeyboardInterrupt: 

### 7. Read forward Key Financial Statistic data from Yahoo Finance files 

In [18]:
import os
import re
from datetime import datetime
from time import mktime
#import datetime
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

directory_pro_his_fin_data = os.path.join(os.getcwd(), 'Processed_Historical_Finance_Data')
avaiable_tickers = [os.path.splitext(x)[0] for x in os.listdir(directory_pro_his_fin_data)]

data_path = os.path.join(os.path.join(os.getcwd(), 'Data'), "Forward.csv")
directory_fwd_fin_data = os.path.join(os.getcwd(), 'Forward_Finance_Data_Web_Files')

key_features = [
        'Market Cap',
        'Enterprise Value',
        'Trailing P/E',
        'Forward P/E',
        'PEG Ratio',
        'Price/Sales',
        'Price/Book',
        'Enterprise Value/Revenue',
        'Enterprise Value/EBITDA',
        'Profit Margin',
        'Operating Margin',
        'Return on Assets',
        'Return on Equity',
        'Revenue',
        'Revenue Per Share',
        'Gross Profit',
        'EBITDA',
        'Net Income Avi to Common',
        'Diluted EPS',
        'Total Cash',
        'Total Cash Per Share',
        'Total Debt',
        'Total Debt/Equity',
        'Current Ratio',
        'Book Value Per Share',
        'Operating Cash Flow|From Operations',
        'Levered Free Cash Flow|Free Cashflow',
        'Beta',
        '% Held by Insiders',
        '% Held by Institutions',
        'Shares Short',
        'Short Ratio',
        'Short % of Float',
        'Shares Short',
        'Trailing Annual Dividend Rate|Annual Dividend',
        'Trailing Annual Dividend Yield|Dividend Yield'
    ]

#Run through each S&P 500 companies to read the Yahoo Finance offline file
counter = 0
key_features_lst = []
for index, row in df_sp500_comp.iterrows():
    try: 
        ticker_symbol = row['Ticker symbol'].replace('.','-')              
        #ticker_folder = os.path.join(directory_pro_his_fin_data, ticker_symbol.lower())
        security = row['Security']
        sector = row['GICS Sector']
        industry = row['GICS Sub Industry']        
        
        counter+=1
        print(counter)
        print("Reading data for Ticker [{0}]".format(ticker_symbol))
        
        if ticker_symbol in avaiable_tickers:            
            full_file_path = os.path.join(directory_fwd_fin_data, ticker_symbol+".html")

            null_value_counter = 0
            key_features_value = {}
            
            date_stamp = None
            unix_time = None

            key_features_value['Date'] = None
            key_features_value['Unix_Time'] = None
            key_features_value['Ticker'] = ticker_symbol
            key_features_value['Security'] = security
            key_features_value['GICS Sector'] = sector
            key_features_value['GICS Sub Industry'] = industry
            key_features_value['Stock Price'] = None
            key_features_value['Stock Price Change'] = None
            key_features_value['S&P 500 Index'] = None
            key_features_value['S&P 500 Index Change'] = None
            key_features_value['Stock Price vs S&P 500 Index'] = None
            key_features_value['Ranking'] = None

            html_doc = open(full_file_path, "r").read()
            soup = BeautifulSoup(html_doc, 'lxml')

            #retrieve value of Key Statistic  
            for key in key_features:
                value = None
                tag = soup.find("span", string=re.compile("^" + key + "+"))                  
                if tag != None:      
                    value = tag.parent.next_sibling.string.strip().replace(",","").replace(" ", "").replace("N/A","").upper()
                else:
                    print("!!!DATA NOT AVAIALBLE for KEY[{0}]".format(key),ticker_symbol)
#                     for tag in soup.find_all("td", class_="yfnc_tablehead1"):                            
#                         this_tag = tag.find(string=re.compile("^" + key + "+"))
#                         if this_tag != None:
#                             value = tag.find_next_sibling("td").string.strip().replace(",","").replace(" ", "").replace("N/A","")
#                             break

                if value != None: 
                    if "B" in value:
                        value = float(value.replace("B",""))*1000000000
                    elif "M" in value:
                        value = float(value.replace("M",""))*1000000
                    elif "K" in value:
                        value = float(value.replace("K",""))*1000
                    elif "%" in value:
                        value = float(value.replace("%",""))/100
                    elif value == "":
                        value = None
                    else:
                        value = float(value)
                elif value == '':
                    value = None                   

                if value == None:
                    null_value_counter += 1

                key_features_value[key] = value

            if null_value_counter <= 5:
                key_features_lst.append(key_features_value.copy())
                    
        else:
            print("[{0}-{1}] not a S&P 500 company previously!".format(row['Ticker symbol'].replace('.','-'), row['Security']))
             
    except Exception as e:
        print(str(e))   

        
df = pd.DataFrame([{
        'Ranking':key_features_value.get('Ranking'),
        ###########
        'Date':key_features_value.get('Date'),
        'Unix_Time':key_features_value.get('Unix_Time'),
        'Ticker':key_features_value.get('Ticker'),
        'Security':key_features_value.get('Security'),
        'GICS Sector':key_features_value.get('GICS Sector'),
        'GICS Sub Industry':key_features_value.get('GICS Sub Industry'),
        'Stock Price':key_features_value.get('Stock Price'),
        'Stock Price Change':key_features_value.get('Stock Price Change'),
        'S&P 500 Index':key_features_value.get('S&P 500 Index'),
        'S&P 500 Index Change':key_features_value.get('S&P 500 Index Change'),
        'Stock Price vs S&P 500 Index':key_features_value.get('Stock Price vs S&P 500 Index'),
        #############
        'Market Cap':key_features_value.get('Market Cap'),
        'Enterprise Value':key_features_value.get('Enterprise Value'),
        'Trailing P/E':key_features_value.get('Trailing P/E'),
        'Forward P/E':key_features_value.get('Forward P/E'),
        'PEG Ratio':key_features_value.get('PEG Ratio'),
        'Price/Sales':key_features_value.get('Price/Sales'),
        'Price/Book':key_features_value.get('Price/Book'),
        'Enterprise Value/Revenue':key_features_value.get('Enterprise Value/Revenue'),
        'Enterprise Value/EBITDA':key_features_value.get('Enterprise Value/EBITDA'),
        'Profit Margin':key_features_value.get('Profit Margin'),
        'Operating Margin':key_features_value.get('Operating Margin'),
        'Return on Assets':key_features_value.get('Return on Assets'),
        'Return on Equity':key_features_value.get('Return on Equity'),
        'Revenue':key_features_value.get('Revenue'),
        'Revenue Per Share':key_features_value.get('Revenue Per Share'),
        'Gross Profit':key_features_value.get('Gross Profit'),
        'EBITDA':key_features_value.get('EBITDA'),
        'Net Income Avl to Common':key_features_value.get('Net Income Avi to Common'),
        'Diluted EPS':key_features_value.get('Diluted EPS'),
        'Total Cash':key_features_value.get('Total Cash'),
        'Total Cash Per Share':key_features_value.get('Total Cash Per Share'),
        'Total Debt':key_features_value.get('Total Debt'),
        'Total Debt/Equity':key_features_value.get('Total Debt/Equity'),
        'Current Ratio':key_features_value.get('Current Ratio'),
        'Book Value Per Share':key_features_value.get('Book Value Per Share'),
        'Operating Cash Flow':key_features_value.get('Operating Cash Flow|From Operations'),
        'Levered Free Cash Flow':key_features_value.get('Levered Free Cash Flow|Free Cashflow'),
        'Beta':key_features_value.get('Beta'),
        '% Held by Insiders':key_features_value.get('% Held by Insiders'),
        '% Held by Institutions':key_features_value.get('% Held by Institutions'),
        'Shares Short':key_features_value.get('Shares Short'),
        'Short Ratio':key_features_value.get('Short Ratio'),
        'Short % of Float':key_features_value.get('Short % of Float'),
        'Shares Short':key_features_value.get('Shares Short'),
        'Trailing Annual Dividend Rate':key_features_value.get('Trailing Annual Dividend Rate|Annual Dividend'),
        'Trailing Annual Dividend Yield':key_features_value.get('Trailing Annual Dividend Yield|Dividend Yield')
    } for key_features_value in key_features_lst],
    columns = [
            'Ranking',
            ##########
            'Date',
            'Unix_Time',
            'Ticker',
            'Security',
            'GICS Sector',
            'GICS Sub Industry',
            'Stock Price',
            'Stock Price Change',
            'S&P 500 Index',
            'S&P 500 Index Change',
            'Stock Price vs S&P 500 Index',
            ############
            'Market Cap',
            'Enterprise Value',
            'Trailing P/E',
            'Forward P/E',
            'PEG Ratio',
            'Price/Sales',
            'Price/Book',
            'Enterprise Value/Revenue',
            'Enterprise Value/EBITDA',
            'Profit Margin',
            'Operating Margin',
            'Return on Assets',
            'Return on Equity',
            'Revenue',
            'Revenue Per Share',
            'Gross Profit',
            'EBITDA',
            'Net Income Avl to Common',
            'Diluted EPS',
            'Total Cash',
            'Total Cash Per Share',
            'Total Debt',
            'Total Debt/Equity',
            'Current Ratio',
            'Book Value Per Share',
            'Operating Cash Flow',
            'Levered Free Cash Flow',
            'Beta',
            '% Held by Insiders',
            '% Held by Institutions',
            'Shares Short',
            'Short Ratio',
            'Short % of Float',
            'Shares Short',
            'Trailing Annual Dividend Rate',
            'Trailing Annual Dividend Yield'
        ])

df.to_csv(data_path, index=False)
print("Forward Financial Data saved!")

1
Reading data for Ticker [MMM]
2
Reading data for Ticker [ABT]
3
Reading data for Ticker [ABBV]
4
Reading data for Ticker [ACN]
5
Reading data for Ticker [ATVI]
6
Reading data for Ticker [AYI]
[AYI-Acuity Brands Inc] not a S&P 500 company previously!
7
Reading data for Ticker [ADBE]
8
Reading data for Ticker [AMD]
9
Reading data for Ticker [AAP]
[AAP-Advance Auto Parts] not a S&P 500 company previously!
10
Reading data for Ticker [AES]
11
Reading data for Ticker [AET]
12
Reading data for Ticker [AMG]
[AMG-Affiliated Managers Group Inc] not a S&P 500 company previously!
13
Reading data for Ticker [AFL]
14
Reading data for Ticker [A]
15
Reading data for Ticker [APD]
16
Reading data for Ticker [AKAM]
17
Reading data for Ticker [ALK]
[ALK-Alaska Air Group Inc] not a S&P 500 company previously!
18
Reading data for Ticker [ALB]
[ALB-Albemarle Corp] not a S&P 500 company previously!
19
Reading data for Ticker [ARE]
[ARE-Alexandria Real Estate Equities Inc] not a S&P 500 company previously!
2

172
Reading data for Ticker [EA]
173
Reading data for Ticker [EMR]
174
Reading data for Ticker [ETR]
175
Reading data for Ticker [EVHC]
[EVHC-Envision Healthcare] not a S&P 500 company previously!
176
Reading data for Ticker [EOG]
177
Reading data for Ticker [EQT]
178
Reading data for Ticker [EFX]
179
Reading data for Ticker [EQIX]
[EQIX-Equinix] not a S&P 500 company previously!
180
Reading data for Ticker [EQR]
181
Reading data for Ticker [ESS]
[ESS-Essex Property Trust, Inc.] not a S&P 500 company previously!
182
Reading data for Ticker [EL]
183
Reading data for Ticker [ES]
[ES-Eversource Energy] not a S&P 500 company previously!
184
Reading data for Ticker [RE]
[RE-Everest Re Group Ltd.] not a S&P 500 company previously!
185
Reading data for Ticker [EXC]
186
Reading data for Ticker [EXPE]
187
Reading data for Ticker [EXPD]
188
Reading data for Ticker [ESRX]
189
Reading data for Ticker [EXR]
[EXR-Extra Space Storage] not a S&P 500 company previously!
190
Reading data for Ticker [XOM

350
Reading data for Ticker [NUE]
351
Reading data for Ticker [NVDA]
352
Reading data for Ticker [ORLY]
353
Reading data for Ticker [OXY]
354
Reading data for Ticker [OMC]
355
Reading data for Ticker [OKE]
356
Reading data for Ticker [ORCL]
357
Reading data for Ticker [PCAR]
358
Reading data for Ticker [PKG]
[PKG-Packaging Corporation of America] not a S&P 500 company previously!
359
Reading data for Ticker [PH]
360
Reading data for Ticker [PDCO]
361
Reading data for Ticker [PAYX]
362
Reading data for Ticker [PYPL]
[PYPL-PayPal] not a S&P 500 company previously!
363
Reading data for Ticker [PNR]
364
Reading data for Ticker [PBCT]
365
Reading data for Ticker [PEP]
366
Reading data for Ticker [PKI]
367
Reading data for Ticker [PRGO]
368
Reading data for Ticker [PFE]
369
Reading data for Ticker [PCG]
370
Reading data for Ticker [PM]
371
Reading data for Ticker [PSX]
372
Reading data for Ticker [PNW]
373
Reading data for Ticker [PXD]
374
Reading data for Ticker [PNC]
375
Reading data for T

# Financial Term:

### Market Cap:
Is the market value of a company's outstanding shares. This figure is found by taking the stock price and multiplying it by the total number of shares outstanding. The classification of companies into different caps also allows investors to gauge the growth versus risk potential. Historically, large caps have experienced slower growth with lower risk. Meanwhile, small caps have experienced higher growth potential, but with higher risk.  

Read more: Market Capitalization Defined https://www.investopedia.com/articles/basics/03/031703.asp#ixzz58xAV93GE 
Follow us: Investopedia on Facebook

### Enterprise Value  (EV):
A measure of a company's total value, often used as a more comprehensive alternative to equity market capitalization. Enterprise value is calculated as the market capitalization plus debt, minority interest and preferred shares, minus total cash and cash equivalents.

EV = market value of common stock + market value of preferred equity + market value of debt + minority interest - cash and investments. 

Read more: Enterprise Value (EV) | Investopedia https://www.investopedia.com/terms/e/enterprisevalue.asp#ixzz58xAHb33d 
Follow us: Investopedia on Facebook

### Trailing Price-To-Earnings - Trailing P/E:

Trailing price-to-earnings (P/E) is calculated by taking the current stock price and dividing it by the trailing earnings per share (EPS) for the past 12 months. Analysts like the P/E ratio because it places a relative price tag on earnings. This relative price tag can be used to look for bargains or to determine when a stock is too expensive. 

Read more: Trailing Price-To-Earnings (Trailing P/E) https://www.investopedia.com/terms/t/trailingpe.asp#ixzz58xA5ZE15 
Follow us: Investopedia on Facebook

### Forward Price To Earnings - Forward P/E:

Forward price to earnings (forward P/E) is a measure of the price-to-earnings (P/E) ratio using forecasted earnings for the P/E calculation. Analysts use earnings estimates to determine what the relative value of the company will be at a future level of earnings. The forward P/E estimates the relative value of the earnings. If the forward P/E ratio is lower than the current P/E ratio, it means analysts are expecting earnings to increase; if the forward P/E is higher than the current P/E ratio, analysts expect a decrease in earnings. 

Read more: Forward Price To Earnings (Forward P/E) https://www.investopedia.com/terms/f/forwardpe.asp#ixzz58xAABlwN 
Follow us: Investopedia on Facebook

### Price/Earnings To Growth - PEG Ratio:

A stock's price-to-earnings (P/E) ratio divided by the growth rate of its earnings for a specified time period. The PEG ratio is used to determine a stock's value while taking the company's earnings growth into account, and is considered to provide a more complete picture than the P/E ratio.

While a low P/E ratio may make a stock look like a good buy, factoring in the company's growth rate to get the stock's PEG ratio can tell a different story. The lower the PEG ratio, the more the stock may be undervalued given its earnings performance. The degree to which a PEG ratio value indicates an over or underpriced stock varies by industry and by company type, though a broad rule of thumb is that a PEG ratio below one is desirable.

Read more: Price/Earnings To Growth (PEG Ratio) https://www.investopedia.com/terms/p/pegratio.asp#ixzz58x9kpIqs 
Follow us: Investopedia on Facebook

### Price-To-Sales Ratio - Price/Sales:

The price-to-sales ratio is a valuation ratio that compares a company’s stock price to its revenues. The price-to-sales ratio is an indicator of the value placed on each dollar of a company’s sales or revenues. It can be calculated either by dividing the company’s market capitalization by its total sales over a 12-month period, or on a per-share basis by dividing the stock price by sales per share for a 12-month period.

Read more: Price-To-Sales Ratio (Price/Sales) https://www.investopedia.com/terms/p/price-to-salesratio.asp#ixzz58xAn7jwu 
Follow us: Investopedia on Facebook

### Price-To-Book Ratio - P/B Ratio:

The price-to-book ratio (P/B Ratio) is a ratio used to compare a stock's market value to its book value. It is calculated by dividing the current closing price of the stock by the latest quarter's book value per share. Also known as the "price-equity ratio".

P/B Ratio = (Market Price per Share / Book Value per Share)

where Book Value per Share =  (Total Assets - Total Liabilities) / Number of shares outstanding

A lower P/B ratio could mean that the stock is undervalued. However, it could also mean that something is fundamentally wrong with the company. As with most ratios, be aware that this varies by industry.

This ratio also gives some idea of whether you're paying too much for what would be left if the company went bankrupt immediately.

Read more: Price-To-Book Ratio (P/B Ratio) https://www.investopedia.com/terms/p/price-to-bookratio.asp#ixzz58xC8lrBw 
Follow us: Investopedia on Facebook

### Enterprise-Value-To-Revenue Multiple - EV/R:

The enterprise-value-to-revenue multiple (EV/R) is a measure of the value of a stock that compares a company's enterprise value to its revenue. EV/R is one of several fundamental indicators that investors use to determine whether a stock is priced well. The EV/R multiple is also often used to determine a company's valuation in the case of a potential acquisition.

Read more: Enterprise-Value-To-Revenue Multiple (EV/R) https://www.investopedia.com/terms/e/ev-revenue-multiple.asp#ixzz58xCxwFGs 
Follow us: Investopedia on Facebook

### EV/EBITDA Multiple:

The value of the metric is determined by dividing a company’s enterprise value (EV) by its earnings before interest, taxes, depreciation and amortization (EBITDA). It is ideal for analysts and potential investors looking to compare companies within the same industry. Typically, EV/EBITDA values below 10 are seen as healthy, but comparison of relative values among firms operating in the same industry is a good way for investors to determine companies with the healthiest EV/EBITDA within a specific sector.

Read more: What is considered a healthy EV/EBITDA? | Investopedia https://www.investopedia.com/ask/answers/072715/what-considered-healthy-evebitda.asp#ixzz58xG25yOc 
Follow us: Investopedia on Facebook

### Profit Margin:

Profit margin is a profitability ratios calculated as net income divided by revenue, or net profits divided by sales. Profit margins are expressed as a percentage and, in effect, measure how much out of every dollar of sales a company actually keeps in earnings.

Read more: Profit Margin https://www.investopedia.com/terms/p/profitmargin.asp#ixzz58xH9Ddzq 
Follow us: Investopedia on Facebook

### Operating Margin'
Operating margin is a margin ratio used to measure a company's pricing strategy and operating efficiency.

Operating margin is a measurement of what proportion of a company's revenue is left over after paying for variable costs of production such as wages, raw materials, etc. It can be calculated by dividing a company’s operating income (also known as "operating profit") during a given period by its net sales during the same period. “Operating income” here refers to the profit that a company retains after removing operating expenses (such as cost of goods sold and wages) and depreciation. “Net sales” here refers to the total value of sales minus the value of returned goods, allowances for damaged and missing goods, and discount sales.

Operating margin gives analysts an idea of how much a company makes (before interest and taxes) on each dollar of sales. Generally speaking, the higher a company’s operating margin is, the better off the company is. If a company's margin is increasing, it is earning more per dollar of sales.

Read more: Operating Margin https://www.investopedia.com/terms/o/operatingmargin.asp#ixzz58xIAdwTB 
Follow us: Investopedia on Facebook

### Return on Assets - ROA:

an indicator of how profitable a company is relative to its total assets. ROA gives a manager, investor, or analyst an idea as to how efficient a company's management is at using its assets to generate earnings. Return on assets is displayed as a percentage and its calculated as:

ROA = Net Income / Total Assets

Sometimes, the ROA is referred to as "return on investment".

In basic terms, ROA tells you what earnings were generated from invested capital (assets). ROA for public companies can vary substantially and will be highly dependent on the industry. This is why when using ROA as a comparative measure, it is best to compare it against a company's previous ROA numbers or against a similar company's ROA.

Read more: Return On Assets (ROA) | Investopedia https://www.investopedia.com/terms/r/returnonassets.asp#ixzz58xKyNsId 
Follow us: Investopedia on Facebook

### Return On Equity - ROE:

The amount of net income returned as a percentage of shareholders equity. Return on equity measures a corporation's profitability by revealing how much profit a company generates with the money shareholders have invested.

ROE is expressed as a percentage and calculated as:

Return on Equity = Net Income/Shareholder's Equity

Net income is for the full fiscal year (before dividends paid to common stock holders but after dividends to preferred stock.) Shareholder's equity does not include preferred shares.

Also known as "return on net worth" (RONW). 

Read more: Return On Equity (ROE) https://www.investopedia.com/terms/r/returnonequity.asp#ixzz58xLTsWUb 
Follow us: Investopedia on Facebook

### Revenue:

Revenue is the amount of money that a company actually receives during a specific period, including discounts and deductions for returned merchandise. It is the top line or gross income figure from which costs are subtracted to determine net income.

Revenue is calculated by multiplying the price at which goods or services are sold by the number of units or amount sold.

Read more: Revenue | Investopedia https://www.investopedia.com/terms/r/revenue.asp#ixzz58xLni6hB 
Follow us: Investopedia on Facebook

### Gross Profit:

Gross profit is the profit a company makes after deducting the costs associated with making and selling its products, or the costs associated with providing its services. Gross profit will appear on a company's income statement, and can be calculated with this formula:

Gross profit = Revenue - Cost of Goods Sold

Read more: Gross Profit | Investopedia https://www.investopedia.com/terms/g/grossprofit.asp#ixzz58xM0uqd4 
Follow us: Investopedia on Facebook

### EBITDA:

EBITDA (Earnings Before Interest, Taxes, Depreciation and Amortization) is a measure computed for a company that looks at its "top line" earnings before deducting interest expense, taxes and depreciation charges. It is used to compare related companies and for purposes of valuation.

Read more: Adjusted EBITDA Definition | Investopedia https://www.investopedia.com/terms/a/adjusted-ebitda.asp#ixzz58xMjJRzv 
Follow us: Investopedia on Facebook

### Net Income - NI:

Net income (NI) is a company's total earnings (or profit); net income is calculated by taking revenues and subtracting the costs of doing business such as depreciation, interest, taxes and other expenses. This number appears on a company's income statement and is an important measure of how profitable the company is over a period of time.

Businesses use net income to calculate their earnings per share (EPS). Business analysts often refer to net income to as the bottom line, since it is listed at the bottom of the income statement. 

Read more: Net Income (NI) https://www.investopedia.com/terms/n/netincome.asp#ixzz58xNJxz4k 
Follow us: Investopedia on Facebook

### Diluted Earnings Per Share - Diluted EPS:

Diluted EPS is a performance metric used to gauge the quality of a company's earnings per share (EPS) if all convertible securities were exercised. Convertible securities are all outstanding convertible preferred shares, convertible debentures, stock options (primarily employee-based) and warrants. Unless the company has no additional potential shares outstanding, which is a relatively rare circumstance, the diluted EPS will always be lower than the simple EPS.

Read more: Diluted Earnings Per Share (Diluted EPS) https://www.investopedia.com/terms/d/dilutedeps.asp#ixzz58xNqChpM 
Follow us: Investopedia on Facebook

### Net Cash:

Net cash is a company's total cash minus total liabilities when discussing financial statements. Net cash is commonly used in evaluating a company's cash flow, and can refer to the amount of cash remaining after a transaction has been completed once all charges and deductions related to the transaction have been subtracted.

Read more: Net Cash https://www.investopedia.com/terms/n/net-cash.asp#ixzz58xOGswRv 
Follow us: Investopedia on Facebook

### Cash Per Share:

A company's total cash divided by its shares outstanding. Cash per share is the percentage of a firm's share price that is immediately accessible for spending on activities such as research and development, mergers and acquisitions, purchasing assets, paying down debt, buying back shares and making dividend payments to shareholders. Cash per share consists of cash and short-term investments. It is money that a firm has on hand, it does not come from borrowing or financing activities.

When a firm has high levels of cash per share, it is holding a significant percentage of its assets in a very liquid form. This decision can indicate economic uncertainty and an unwillingness by firms to invest given the current economic climate. High levels of cash per share can indicate that a firm is performing well, with positive earnings and cash flow and the ability to reinvest in itself; however, high levels of cash per share do not always coincide with overall financial strength.

Read more: Cash Per Share https://www.investopedia.com/terms/c/cash-per-share.asp#ixzz58xOcm9Hb 
Follow us: Investopedia on Facebook

### Debt/Equity Ratio:

Debt/Equity (D/E) Ratio, calculated by dividing a company’s total liabilities by its stockholders' equity, is a debt ratio used to measure a company's financial leverage. The D/E ratio indicates how much debt a company is using to finance its assets relative to the value of shareholders’ equity. The formula for calculating D/E ratios is:

Debt/Equity Ratio = Total Liabilities / Shareholders' Equity

The debt/equity ratio is also referred to as a risk or gearing ratio.

Read more: Debt/Equity Ratio https://www.investopedia.com/terms/d/debtequityratio.asp
Follow us: Investopedia on Facebook

### Book Value Per Common Share:

A measure used by owners of common shares in a firm to determine the level of safety associated with each individual share after all debts are paid accordingly.

Book value per share = (Total Shareholder Equity - Preferred Equity)/Total Outstanding Shares

Should the company decide to dissolve, the book value per common indicates the dollar value remaining for common shareholders after all assets are liquidated and all debtors are paid.

In simple terms it would be the amount of money that a holder of a common share would get if a company were to liquidate.

Read more: Book Value Per Common Share https://www.investopedia.com/terms/b/bookvaluepercommon.asp#ixzz58yVjXYtu 
Follow us: Investopedia on Facebook

### Operating Cash Flow - OCF:

A measure of the amount of cash generated by a company's normal business operations. Operating cash flow indicates whether a company is able to generate sufficient positive cash flow to maintain and grow its operations, or it may require external financing for capital expansion. Generally accepted accounting principles (GAAP) require public companies to calculate operating cash flow using an indirect method by adjusting net income to cash basis using changes in non-cash accounts, such as depreciation, accounts receivable and changes in inventory.

Read more: Operating Cash Flow (OCF) https://www.investopedia.com/terms/o/operatingcashflow.asp#ixzz58yWjNMZp 
Follow us: Investopedia on Facebook

### Levered Free Cash Flow:

Levered free cash flow is the amount of cash a company has left remaining after paying all its financial obligations. Levered free cash flow is important to both investors and company management, because it is the amount of cash that a company can use to pay dividends to shareholders and/or to make investments in growing the company's business. The amount of levered cash flow a company has can be negative even though operating cash flow is positive, if the amount of operating cash flow is insufficient to cover all of the company's financial obligations.

Read more: Levered Free Cash Flow https://www.investopedia.com/terms/l/levered-free-cash-flow.asp#ixzz58yX34Sh7 
Follow us: Investopedia on Facebook

### Beta:

Beta is a measure of a stock's volatility in relation to the market. By definition, the market has a beta of 1.0, and individual stocks are ranked according to how much they deviate from the market. A stock that swings more than the market over time has a beta above 1.0. If a stock moves less than the market, the stock's beta is less than 1.0. High-beta stocks are supposed to be riskier but provide a potential for higher returns; low-beta stocks pose less risk but also lower returns.  

Read more: Beta https://www.investopedia.com/terms/b/beta.asp#ixzz58x9tcVob 
Follow us: Investopedia on Facebook

### Closely Held Stock:

Closely held stock is a circumstance where a company’s common shares are predominantly owned by one individual owner or by a small group of controlling stockholders. This is in contrast to a widely held stock, in which thousands or even millions of different investors may own shares in a large company.

Read more: Closely Held Stock https://www.investopedia.com/terms/c/closely-held-stock.asp#ixzz58yXVmtaD 
Follow us: Investopedia on Facebook

### Shares Short:

A short sale is a transaction in which an investor sells borrowed securities in anticipation of a price decline and is required to return an equal number of shares at some point in the future. A short seller makes money if the stock goes down in price, while a long position makes money when the stock goes up. In real estate, a short sale means selling a house for less than the mortgage owed with the lender's approval.

Read more: Short Sale https://www.investopedia.com/terms/s/shortsale.asp#ixzz58yY5McaH 
Follow us: Investopedia on Facebook

### Short Interest Ratio:

The short interest ratio is a sentiment indicator that is derived by dividing the short interest by the average daily volume for a stock. Also known as the days to cover ratio, it is used by both fundamental and technical traders to identify the prevailing sentiment the market has for a specific stock.

Shorting a stock is the opposite of buying a stock. Specifically, it is the selling of a stock that is not owned by the investor. The goal of selling a stock short is to make money when it falls to a lower price. The investor is betting that the price of the stock will go down. Once the price goes down, the investor can buy the stock at the lower price without having to own the stock. The expectation is for the price of the stock to go down. Looked at in aggregate, short interest and the short interest ratio can provide clues about the level of bearish sentiment in the market.

Read more: Short Interest Ratio https://www.investopedia.com/terms/s/shortinterestratio.asp#ixzz58yZ1Pr1h 
Follow us: Investopedia on Facebook

### Dividend Rate:

The dividend rate is the total amount of the expected dividend payments from an investment, fund or portfolio expressed on an annualized basis plus any additional non-recurring dividends that may be received during that period. Depending on the company's preferences and strategy, the dividend rate can be fixed or adjustable.

Dividend Rate = (recent dividend)x(# of dividend periods per year)+extra dividends

Read more: Dividend Rate https://www.investopedia.com/terms/d/dividendrate.asp#ixzz58yZxMf6D 
Follow us: Investopedia on Facebook

### Dividend Yield:

A stock's dividend yield is expressed as an annual percentage and is calculated as the company's annual cash dividend per share divided by the current price of the stock. The dividend yield is found in the stock quotes of dividend-paying companies. Investors should note that stock quotes record the per share dollar amount of a company's latest quarterly declared dividend. This quarterly dollar amount is annualized and compared to the current stock price to generate the per annum dividend yield, which represents an expected return.

Read more: Investment Valuation Ratios: Dividend Yield https://www.investopedia.com/university/ratios/investment-valuation/ratio7.asp#ixzz58yaUhHuo 
Follow us: Investopedia on Facebook
